In [1]:
!pip install -U openai

import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    input="Write a one-sentence bedtime story about a unicorn."
)

print(response.output_text)

Under a sky of sleepy stars, a silver-maned unicorn tiptoed through moonlit meadows, gathering children’s dreams in her velvet satchel and humming them into lullabies that floated home on the night breeze.


In [3]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    text_format=CalendarEvent,
)

event = response.output_parsed
print(event)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


In [4]:
from openai import OpenAI
from pydantic import BaseModel

client = OpenAI()

class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {
            "role": "system",
            "content": "You are a helpful math tutor. Guide the user through the solution step by step.",
        },
        {"role": "user", "content": "how can I solve 8x + 7 = -23"},
    ],
    text_format=MathReasoning,
)

math_reasoning = response.output_parsed
print(math_reasoning)

steps=[Step(explanation='Start with the equation: \\(8x + 7 = -23\\). Your goal is to solve for \\(x\\).', output='8x + 7 = -23'), Step(explanation='Subtract 7 from both sides to begin isolating the variable \\(x\\).', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplify both sides of the equation.', output='8x = -30'), Step(explanation='Divide both sides by 8 to solve for \\(x\\).', output='\\(x = \\frac{-30}{8}\\)'), Step(explanation='Simplify the fraction by dividing the numerator and the denominator by their greatest common divisor, which is 2.', output='\\(x = \\frac{-15}{4}\\)')] final_answer='\\(x = \\frac{-15}{4}\\)'


In [5]:
from openai import OpenAI
import json

client = OpenAI()

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))

            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseReasoningItem(id='rs_0ce071371117a5da0068ff0b77a1fc819e9cc6f688c0627193', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_kcQTprTxIjczm6QeYWD4w42m', name='get_horoscope', type='function_call', id='fc_0ce071371117a5da0068ff0b7a7cf0819ea96bba6fa11c56b3', status='completed'), {'type': 'function_call_output', 'call_id': 'call_kcQTprTxIjczm6QeYWD4w42m', 'output': '{"horoscope": "{\'sign\': \'Aquarius\'}: Next Tuesday you will befriend a baby otter."}'}]
Final output:
{
  "id": "resp_0ce071371117a5da0068ff0b7debc0819e8a6e5c00ff78f1ec",
  "created_at": 1761545086.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "msg_0ce07